# Free-for-all practice file, not included in final submission

In [20]:
import math

from bloqade import qasm2
from kirin.dialects import ilist

In [21]:
def ghz_linear(n: int):
    n_qubits = int(2**n)

    @qasm2.extended
    def ghz_linear_program():

        qreg = qasm2.qreg(n_qubits)
        # Apply a Hadamard on the first qubit
        qasm2.h(qreg[0])
        # Create a cascading sequence of CX gates
        # necessary for quantum computers that
        # only have nearest-neighbor connectivity between qubits
        for i in range(1, n_qubits):
            qasm2.cx(qreg[i - 1], qreg[i])

    return ghz_linear_program

In [22]:
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

target = QASM2()
ast = target.emit(ghz_linear(2))
pprint(ast)

OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[4];
h qreg[0];
CX qreg[0], qreg[1];
CX qreg[1], qreg[2];
CX qreg[2], qreg[3];
